In [2]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

In [3]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes"
# folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
folders = ['webd-dependency', 'combined-fasttext', 'essays-dependency',
          'original', 'essays-fasttext', 'ibm-fasttext', 'webd-fasttext']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            if subdir.endswith('100'):
                print(path)
                indexref = pt.IndexRef.of(path)
                index = pt.IndexFactory.of(indexref)
                dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
                tuned_dlm, parameters = pt.KFoldGridSearch(
                    dlm,
                    {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                    [topics20, topics21],
                    [qrels20, qrels21],
                    objective
                )
                
                print(pt.Experiment([tuned_dlm], topics, qrels, names=[folders[i] + '_tuned_dlm_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_dlm], topics20, qrels, names=[folders[i] + '_tuned_dlm_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_dlm], topics21, qrels, names=[folders[i] + '_tuned_dlm_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

                print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm__unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm__unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                tuned_dlm.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_dlm.csv")
                i += 1
            

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties
Fold 1
Best ndcg_cut_5 is 0.387056
Best setting is ['BR(DirichletLM) dirichletlm.mu=750']
Fold 2
Best ndcg_cut_5 is 0.344717
Best setting is ['BR(DirichletLM) dirichletlm.mu=1750']


/tmp/ipykernel_3689045/2604357178.py:29: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics, qrels, names=[folders[i] + '_tuned_dlm_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-dependency\_tuned\_dlm\_ &    0.349104 &     0.259055 &  0.411606 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689045/2604357178.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics20, qrels, names=[folders[i] + '_tuned_dlm_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  webd-dependency\_tuned\_dlm\_20 &    0.343726 &     0.262526 &  0.46289 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                      name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_ &     0.32813 &     0.241347 &  0.401374 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689045/2604357178.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics20, qrels, names=[folders[i] + '_tuned_dlm_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_20 &    0.266989 &     0.207011 &  0.434627 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3689045/2604357178.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics21, qrels, names=[folders[i] + '_tuned_dlm_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_3689045/2604357178.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm__unjudged'], eval_metrics=["ndcg_cut_5

\begin{tabular}{llrrr}
\toprule
{} &                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_21 &    0.388048 &     0.274997 &  0.368786 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                               name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_\_unjudged &     0.58949 &     0.473152 &  0.401374 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_\_unjudged20 &     0.68873 &     0.531272 &  0.434627 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_dlm\_\_unjudged21 &     0.57304 &     0.493779 &  0.368786 \\
\bottomrule
\end{tabular}



In [5]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print(path)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )

            print(pt.Experiment([tuned_dlm], topics, qrels, names=[folders[i] + '_tuned_dlm_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm], topics20, qrels, names=[folders[i] + '_tuned_dlm_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm], topics21, qrels, names=[folders[i] + '_tuned_dlm_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm__unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm__unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            i += 1
            

/workspace/src/grid_search_right_qrels/super_folder//data.properties
Fold 1
Best ndcg_cut_5 is 0.455074
Best setting is ['BR(DirichletLM) dirichletlm.mu=1000']
Fold 2
Best ndcg_cut_5 is 0.250555
Best setting is ['BR(DirichletLM) dirichletlm.mu=1500']


/tmp/ipykernel_2391636/1037287187.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics, qrels, names=[folders[i] + '_tuned_dlm_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_ &     0.34844 &     0.311509 &  0.610536 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391636/1037287187.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics20, qrels, names=[folders[i] + '_tuned_dlm_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_20 &    0.247279 &     0.252619 &  0.601926 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391636/1037287187.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm], topics21, qrels, names=[folders[i] + '_tuned_dlm_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391636/1037287187.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm__unjudged'], eval_metrics=["ndcg_cut_5

\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_21 &    0.447577 &      0.36922 &  0.618973 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_\_unjudged &     0.59213 &     0.608695 &  0.610536 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                    name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_\_unjudged20 &    0.606138 &     0.625352 &  0.601926 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                    name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_\_unjudged21 &    0.601868 &     0.638206 &  0.618973 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391636/1037287187.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391636/1037287187.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_